In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import pandas as pd

def scrape_prismatic_evolutions_selenium(output_csv: str = "./output/prismatic_evolutions.csv"):
    url = "https://www.pricecharting.com/console/pokemon-prismatic-evolutions"

    # 1. Setup headless Chrome
    chrome_opts = Options()
    chrome_opts.add_argument("--headless")
    chrome_opts.add_argument("--disable-gpu")
    chrome_opts.add_argument("--no-sandbox")

    service = Service("/opt/homebrew/bin/chromedriver")
    driver = webdriver.Chrome(service=service, options=chrome_opts)

    try:
        # 2. Load the page
        driver.get(url)

        # 3. Wait until the JS‑rendered table with id="games_table" is present
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.ID, "games_table"))
        )

        # 4. Grab the fully rendered page source
        html = driver.page_source

    finally:
        driver.quit()

    # 5. Parse with BeautifulSoup and find the table by ID
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table", id="games_table")
    if table is None:
        raise RuntimeError("Could not find table with id='games_table' on the page.")

    # 6. Extract data rows
    rows = []
    # Assume first <tr> is header
    for tr in table.find_all("tr")[1:]:
        cols = tr.find_all("td")
        if len(cols) < 4:
            continue

        card_name = cols[1].get_text(strip=True)
        ungraded  = cols[2].get_text(strip=True)
        grade9     = cols[3].get_text(strip=True)
        psa10      = cols[4].get_text(strip=True)

        # Optional: grab detail link
        link = cols[0].find("a", href=True)
        detail_url = link["href"] if link else ""

        rows.append({
            "card":       card_name,
            "ungraded":   ungraded,
            "grade_9":    grade9,
            "psa_10":     psa10,
            "detail_url": detail_url
        })

    # 7. Build DataFrame and save to CSV
    df = pd.DataFrame(rows)
    
    parts = df['card'].str.extract(r'^(.*?)\s*#\s*(\d+)\s*$', expand=True)
    df['card_name']   = parts[0].fillna(df['card'])
    df['card_number'] = parts[1].fillna('')
    df = df.drop(columns=['card'])

    cols = ['card_name','card_number'] + [c for c in df.columns if c not in ('card_name','card_number')]
    df = df[cols]
    
    df.to_csv(output_csv, index=False)
    print(f"Scraped {len(df)} rows → '{output_csv}'")

    return df


In [23]:
df = scrape_prismatic_evolutions_selenium()
print(df.head())

Scraped 50 rows → './output/prismatic_evolutions.csv'
               card_name card_number   ungraded    grade_9     psa_10  \
0             Umbreon ex         161  $1,164.04  $1,212.50  $3,075.00   
1             Sylveon ex         156    $410.99    $410.50  $1,010.71   
2  Umbreon [Master Ball]          59    $137.29    $161.25    $642.93   
3             Pikachu ex         179     $67.60     $86.45    $298.00   
4             Leafeon ex         144    $349.69    $342.10    $783.01   

                                          detail_url  
0  https://www.pricecharting.com/game/pokemon-pri...  
1  https://www.pricecharting.com/game/pokemon-pri...  
2  https://www.pricecharting.com/game/pokemon-pri...  
3  https://www.pricecharting.com/game/pokemon-pri...  
4  https://www.pricecharting.com/game/pokemon-pri...  
